In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [3]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [4]:
print(f'X_train.shape:{X_train.shape}, X_test.shape:{X_test.shape}, y_train.shape:{y_train.shape}, y_test.shape:{y_test.shape}')

X_train.shape:(109820, 16, 3), X_test.shape:(27455, 16, 3), y_train.shape:(109820,), y_test.shape:(27455,)


In [5]:
# Define the MLP model
class MyMLP(nn.Module):
    def __init__(self, input_size, num_classes=6):
        super(MyMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Model Initialization
input_size = timestep * X_series.shape[2]  # Calculate input size
model = MyMLP(input_size)

# DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)



In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')

# Train the model
train(model, train_loader, criterion, optimizer)

#
model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/MLP_base.pth"
torch.save(model.state_dict(), model_path)


Epoch 0, Loss: 0.5021717630170442
Epoch 1, Loss: 0.36609900722796823
Epoch 2, Loss: 0.31904657409116105
Epoch 3, Loss: 0.29136974481748534
Epoch 4, Loss: 0.2719781262613399
Epoch 5, Loss: 0.25419779397147796
Epoch 6, Loss: 0.2422728936289956
Epoch 7, Loss: 0.22908410076746644
Epoch 8, Loss: 0.2196793676221565
Epoch 9, Loss: 0.21247534934726559
Epoch 10, Loss: 0.20520914606923685
Epoch 11, Loss: 0.19735367139171817
Epoch 12, Loss: 0.19158412968969984
Epoch 13, Loss: 0.18667062871822424
Epoch 14, Loss: 0.1815280131507222
Epoch 15, Loss: 0.1771163104670795
Epoch 16, Loss: 0.1735728815673375
Epoch 17, Loss: 0.17065010669734546
Epoch 18, Loss: 0.16690869244045659
Epoch 19, Loss: 0.16357700967192998
Epoch 20, Loss: 0.16093596258246295
Epoch 21, Loss: 0.15654324774200526
Epoch 22, Loss: 0.15464142740580378
Epoch 23, Loss: 0.1527086095596796
Epoch 24, Loss: 0.14820801055315005
Epoch 25, Loss: 0.14857439270963793
Epoch 26, Loss: 0.1447292214242138
Epoch 27, Loss: 0.1430949682672671
Epoch 28, Lo

In [7]:
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {total_loss / len(test_loader)}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')

# DataLoader for test set
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Evaluate the model
evaluate(model, test_loader, criterion)


Test set: Average loss: 0.4590757123318377, Accuracy: 25037/27455 (91%)


#### Coreml evaluation

In [22]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for id in range(len(X_test)):
        X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_new})
        pred_class = np.argmax(output_dict['linear_2'])
        predictions.append(pred_class)
    
    accuracy = np.sum(predictions == y_test) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

### Convert to Coreml

In [25]:
import coremltools as ct
example_input = torch.rand(1,16, 3) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
mlp_coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
mlp_coreml_model.save("mlp.mlpackage")

Running MIL backend_mlprogram pipeline: 100%|█████████████████████████████████████████████| 12/12 [00:00<00:00, 7308.21 passes/s]


In [26]:
import coremltools as ct
import coremltools.optimize.coreml as cto

mlp_coreml_model = ct.models.MLModel("mlp.mlpackage")

In [27]:
model_name = mlp_coreml_model
model_path = "mlp.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9117829175013659
Size of the model: 0.12 KB


In [42]:
import os
os.path.getsize("/Users/sandeep/Desktop/BUCourses/Project/light-har/code/" + "mlp.mlpackage")


128

### Palletization example 


In [43]:
# define op config 
op_config = cto.OpPalettizerConfig(mode="kmeans", nbits=6)

# define optimization config by applying the op config globally to all ops 
config = cto.OptimizationConfig(global_config=op_config)

# palettize weights
compressed_mlmodel = cto.palettize_weights(mlmodel, config)

compressed_mlmodel.save("mlp_palletization.mlpackage")

Running compression pass palettize_weights: 100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 121.25 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 11037.64 passes/s]


In [13]:
import coremltools as ct
import coremltools.optimize.coreml as cto

compressed_mlmodel = ct.models.MLModel("mlp_palletization.mlpackage")

In [14]:
model_name = compressed_mlmodel
model_path = "mlp_palletization.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9098160626479694
Size of the model: 0.12 KB


### Pruning in Coreml

#### OpThresholdPrunerConfig: Sets all weight values below a certain value.

In [15]:
from coremltools.optimize.coreml import (
    OpThresholdPrunerConfig,
    OptimizationConfig,
    prune_weights,
)

op_config = OpThresholdPrunerConfig(
    threshold=0.001,
    minimum_sparsity_percentile=0.01,
    weight_threshold=1024,
)
config = OptimizationConfig(global_config=op_config)
pruned_model = prune_weights(mlmodel, config=config)


Running compression pass prune_weights:   0%|                                                            | 0/5 [00:00<?, ? ops/s]weight value has sparsity of 0.0017903645833333333 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.002197265625 < minimum_sparsity_percentile 0.01. Skipped.
Running compression pass prune_weights: 100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00, 5939.26 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 22682.13 passes/s]


In [16]:
pruned_model.save("pruned_mlmodel.mlpackage")

In [17]:
model_name = pruned_model
model_path = "pruned_mlmodel.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9117829175013659
Size of the model: 0.12 KB


#### OpMagnitudePrunerConfig: Prune the weights with a constant sparsity percentile

In [18]:
from coremltools.optimize.coreml import (
    OpMagnitudePrunerConfig,
    OptimizationConfig,
    prune_weights,
)

op_config = OpMagnitudePrunerConfig(
    target_sparsity=0.6,
    weight_threshold=1024,
)
config = OptimizationConfig(global_config=op_config)
magnitude_pruner = prune_weights(mlmodel, config=config)


Running compression pass prune_weights: 100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00, 2313.46 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 16485.96 passes/s]


In [28]:
magnitude_pruner.save("magnitude_pruner.mlpackage")

In [29]:
model_name = magnitude_pruner
model_path = "magnitude_pruner.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.6382808231651794
Size of the model: 0.12 KB


### Quantization

In [30]:
import coremltools.optimize.coreml as cto

op_config = cto.OpLinearQuantizerConfig(mode="linear_symmetric", weight_threshold=512)
config = cto.OptimizationConfig(global_config=op_config)

compressed_8_bit_model = cto.linear_quantize_weights(mlmodel, config=config)


Running compression pass linear_quantize_weights: 100%|████████████████████████████████████████| 5/5 [00:00<00:00, 2041.82 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|█████████████████████████████████████████████| 12/12 [00:00<00:00, 7233.64 passes/s]


In [31]:
compressed_8_bit_model.save("8bitQuantized_mlmodel.mlpackage")

In [32]:
model_name = compressed_8_bit_model
model_path = "8bitQuantized_mlmodel.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9114186851211072
Size of the model: 0.12 KB
